# Step 1. Reload files & Import Modules - 匯入模組

## 1-1. Reload files - 重讀檔案

In [1]:
# reload file if you eaited them
import crawler104, config.search_params, async_example, threaded_async_job, jobs104
import data_lake
import importlib
importlib.reload(crawler104)
importlib.reload(async_example)
importlib.reload(threaded_async_job)
importlib.reload(jobs104)
importlib.reload(config.search_params)
importlib.reload(data_lake)

<module 'data_lake' from 'C:\\Users\\Rekam\\Documents\\Jobscan\\data_lake.py'>

## 1-2. Import Modules - 匯入模組

In [2]:
# import library
from crawler104 import Crawler104
from config.search_params import get_filter_params
from data_lake import DataLake

In [3]:
# 快速執行 reload & import
%run main.py

# Step 2. Data Source - 爬蟲抓資料

## 2-1. Web Crawler - 抓取Jobs清單

In [4]:
# custom filter params for search - for yidti
role = {'ro':'全職'}
keyword = {'keyword':"後端工程師 python"}
# area = {'area':['新北市', '台北市', '桃園市', '台中市']}
isnew = {'isnew':'三日內'}
jobexp = {'jobexp':['1年以下', '1-3年']}
# 預設
mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
order = {'order':'日期排序'}
asc = {'asc':'遞減'}
# filter_params = get_filter_params(role, keyword, area, isnew, jobexp, mode, order, asc)
filter_params = get_filter_params(role, keyword, isnew, jobexp, mode, order, asc)

# keywords for filter job again
job_keywords = ('工程','資料','python','data','數據','後端')
# exclude keyword for filter company (about gambling)
company_exclude = ('新加坡商冕創有限公司','新博軟體開發股份有限公司','現觀科技股份有限公司')
# -------------------------------------------
# 根據過濾參數開始爬蟲 & 再次根據keyword過濾jobs與company & 在爬取細節
user = "yidti"
crawler = Crawler104(filter_params, user)
crawler.run(job_keywords, company_exclude)

url: https://www.104.com.tw/jobs/search/?ro=1&keyword=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB+python&isnew=3&jobexp=1%2C3&mode=l&order=16&asc=0


Loading: 100%|██████████| 53/53 [01:12<00:00,  1.37s/page]

載入53頁 | 

載入1558筆資料 | 過濾剩1202筆資料 | 花費 88.03 秒


## 2-2. Web Scraper - 抓取Jobs內容(異步&多線程)

In [5]:
crawler.detail()

Processing jobs: 100%|██████████| 1202/1202 [03:20<00:00,  5.99job/s]

Scraping Details for 1202 Jobs | 花費 200.73 秒


## 2-3. Export Flie - 輸出至Excel

In [6]:
crawler.export_excel()

CSV文件保存成功: output/yidti_2024-02-05.xlsx


# Step 3. Data Lake - 資料存入NoSQL

In [1]:
# 快速執行 reload & import
%run main.py

In [42]:
# output to excel file (job)
# ouput to noSQL (job, company, industry)
# data_Lake = DataLake()
# data_Lake.upload_nosql(user, crawler)

NameError: name 'df' is not defined

# Step 4 - Data Warehouse

In [25]:
# create db
import mysql.connector

def createDB():
    db = mysql.connector.connect(
        host="localhost",
        user = "root",
        password = "Sql@1031",
        port = 3306
    )
    db_name = "job_db"
    cursor = db.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    cursor.execute("SHOW DATABASES")
    
    for x in cursor:
      print(x)

    db.commit()
    db.close()

createDB()

('information_schema',)
('job_db',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


In [46]:
import os, time
from dotenv import load_dotenv
load_dotenv()
os.getenv("MODE")
from sqlalchemy import create_engine, text
from urllib.parse import quote

# 對密碼進行 URL 編碼
user = "root"
password = quote("Sql@1031")
host = "localhost"
port = 3306
db_name = "job_db"
# 構建 MySQL 數據庫連接 URL
connector_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db_name}"
# print(connector_url)
# 創建引擎object
engine = create_engine(connector_url, future = True)

# SQL 語句範例
# sql_statement = "SELECT * FROM your_table"  # 替換成實際的表格名稱
sql_job_table =  """
            CREATE TABLE IF NOT EXISTS JobsInfo  (
            `更新日期` DATE,
            `職缺名稱` VARCHAR(100),
            `公司名稱` VARCHAR(100),
            `工作內容` TEXT,
            `職務類別` INT,
            `工作待遇` VARCHAR(100),
            `工作性質` INT,
            `縣市` INT,
            `上班地點` VARCHAR(100),
            `管理責任` INT,
            `出差外派` INT,
            `上班時段` VARCHAR(100),
            `休假制度` INT,
            `可上班日` INT,
            `需求人數` VARCHAR(50),
            `工作經歷` INT,
            `學歷要求` INT,
            `科系要求` VARCHAR(50),
            `語文條件` VARCHAR(50),
            `擅長工具` VARCHAR(500),
            `工作技能` VARCHAR(500),
            `其他要求` TEXT,
            `連結` TEXT,
            PRIMARY KEY (`連結`(255))
);
        """

with engine.connect() as connection:
    print("test OK")
    connection.execute(text(sql_job_table))


test OK


# Step 5 - Data Application

# Step 99. Test Area - 測試區域

In [ ]:
# custom filter params for search - simon
# role = {'ro':'全職'}
# zone = {'zone': '4,5'}
# indcat = {'indcat':'1001000000,1002000000,1012000000,1014000000'}
# exclude = {'excludeJobKeyword': '軟體,會計,文件整理,行政,百貨,資訊,品管,財務,守衛,技術員,法務,品檢,登打,輪班,無經驗,客戶'}
# mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
# order = {'order':'日期排序'}
# asc = {'asc':'遞減'}
# filter_params = get_filter_params(role, zone, indcat, exclude, mode, order, asc)

In [ ]:
# 如果你在一個 Python script 中執行這樣的程式碼，你可能需要使用 asyncio.run() 函數。
# 在異步程式碼中，如果你使用 asyncio.gather 函數收集異步任務的結果，而這些任務沒有顯式返回值，
# gather 函數將返回一個包含每個異步任務結果的列表，而這些結果通常是 None。

import asyncio

async def async_example(i):
    print(f"Start asynchronous task {i}")
    await asyncio.sleep(1)
    print(f"End asynchronous task {i}")
    return f"Result of task {i}"

# 直接在 Jupyter cell 中執行
tasks = [async_example(i) for i in range(5)]
results = await asyncio.gather(*tasks)
print(results)